In [55]:
import pandas as pd, requests, json
from google.colab import files
#uploaded = files.upload()
df = pd.read_excel(r'/content/test.xlsx')
#trim the first six rows since they have report information
df = df.drop(df.index[0:6])

df.columns = df.iloc[0]
df = df.drop(df.index[0])

#the rest of this comes from here
#https://github.com/gboeing/urban-data-science/blob/master/17-Leaflet-Web-Mapping/leaflet-simple-demo/pandas-to-geojson.ipynb

df['Latitude'] = df['Latitude'].astype(float)
df['Longitude'] = df['Longitude'].astype(float)

# we don't need all those columns - only keep useful ones
useful_cols = ['Longitude', 'Latitude', 'Altitude', 'Speed', 'Time', 'Attributes']
df_subset = df[useful_cols]

def df_to_geojson(df, properties, lat='Latitude', lon='Longitude', alt='Altitude'):
    """
    Turn a dataframe containing point data into a geojson formatted python dictionary
    
    df : the dataframe to convert to geojson
    properties : a list of columns in the dataframe to turn into geojson feature properties
    lat : the name of the column in the dataframe that contains latitude data
    lon : the name of the column in the dataframe that contains longitude data
    """
    
    # create a new python dict to contain our geojson data, using geojson format
    geojson = {'type':'FeatureCollection', 'features':[]}

    # loop through each row in the dataframe and convert each row to geojson format
    for _, row in df.iterrows():
        # create a feature template to fill in
        feature = {'type':'Feature',
                   'properties':{},
                   'geometry':{'type':'Point',
                               'coordinates':[]}}

        # fill in the coordinates
        feature['geometry']['coordinates'] = [row[lon],row[lat],row[alt]]

        # for each column, get the value and add it as a new feature property
        for prop in properties:
            feature['properties'][prop] = row[prop]
        
        # add this feature (aka, converted dataframe row) to the list of features inside our dict
        geojson['features'].append(feature)
    
    return geojson

geojson_dict = df_to_geojson(df, properties=useful_cols)
geojson_str = json.dumps(geojson_dict, indent=2)

# save the geojson result to a file
output_filename = 'data.geojson'
with open(output_filename, 'w') as output_file:
    output_file.write('{}'.format(geojson_str))
    
# how many features did we save to the geojson file?
print('{} geotagged features saved to file'.format(len(geojson_dict['features'])))

Longitude
Latitude
Altitude
Speed
Time
Attributes
3861 geotagged features saved to file
